<h3> <center>Hold 05 - Project group 22 </center> </h3>
<center>Rune Ejnar Bang Lejbølle (nvr889@alumni.ku.dk)</center>
<center>Pedram Bakhtiarifard (lcd842@alumni.ku.dk)</center>

<h1><center>Milestone 2</center></h1>

<h3> <center> Loading the data </center> </h3>

In [1]:
import numpy
import pandas as pd
import re
import pandas.io.sql as psql

# Load from local CSV-file

nrows = 100000

source_data = pd.read_csv("500thousand_rows.csv", index_col=None, nrows=nrows, dtype={"id": "string", "domain": "string", "type": "string", "url": "string", "content": "string", "scraped_at": "string", "inserted_at": "string", "updated_at": "string", "title": "string", "authors": "string", "keywords": "string", "meta_keywords": "string", "meta_description": "string", "tags": "string", "summary": "string", "source": "string"})

In [2]:
from cleantext import constants as cleantext_re

punctuation_pattern = re.compile(r'([!"#$%&\'’()*+,\-–—./:;=?@\[\\\]^_`{}~<>\n\t\r])')

source_data['cleaned_content'] = source_data['content']\
    .str.replace(cleantext_re.EMAIL_REGEX, '|email|')\
    .str.replace(cleantext_re.URL_REGEX, '|url|')\
    .str.replace(cleantext_re.NUMBERS_REGEX, '|num|')\
    .str.replace(punctuation_pattern, "")\
    .str.lower()

In [3]:
import nltk
from concurrent.futures import ProcessPoolExecutor

source_data = source_data.dropna(subset=['cleaned_content'])

nltk.download('stopwords')

tokenizer = nltk.RegexpTokenizer(r'(\w+|\|\w+\|)')

source_data['tokenized_content'] = source_data['cleaned_content'].apply(tokenizer.tokenize)

stopwords = nltk.corpus.stopwords.words('english')


def filter_stopwords(tokens):
    return [x for x in tokens if not x in stopwords]


source_data['nostop_content'] = source_data['tokenized_content'].apply(filter_stopwords)

stemmer = nltk.stem.PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]


with ProcessPoolExecutor(max_workers=4) as executor:
    input = source_data['nostop_content'].to_numpy()
    output = numpy.empty_like(input)

    results = executor.map(stem_tokens, input)

    for i, result in enumerate(results):
        output[i] = result

    source_data['stemmed_content'] = output

source_data['joined_content'] = source_data['stemmed_content'].str.join(' ')

[nltk_data] Downloading package stopwords to /home/ped/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
array_pattern = re.compile(r'[^, \[\]\']+(?: [^, \[\]\']+)*')
list_pattern = re.compile(r'[^, ]+(?: [^, ]+)*')

def make_list_atomic(source_data, key, to_split, pattern):
    source_data[to_split + '_split'] = source_data[to_split].str.lower().str.findall(pattern)
    df = source_data[[key, to_split + '_split']][source_data[to_split + '_split'].notnull()]\
        .explode(to_split + '_split', ignore_index=True)
    return df[df[to_split + '_split'].notnull()]

database_authors = make_list_atomic(source_data, 'id', 'authors', list_pattern).drop_duplicates()
database_keywords = make_list_atomic(source_data, 'id', 'keywords', list_pattern).drop_duplicates()
database_meta_keywords = make_list_atomic(source_data, 'id', 'meta_keywords', array_pattern).drop_duplicates()
database_tags = make_list_atomic(source_data, 'id', 'tags', list_pattern).drop_duplicates()

database_articles = source_data[['id', 'domain', 'type', 'url', 'joined_content',
                                 'scraped_at', 'inserted_at', 'updated_at', 'title', 'meta_description', 'summary']]

In [5]:
import psycopg2

conn_string = 'postgres://postgres@localhost/fakenews_data'

pg_conn = psycopg2.connect(conn_string)

cur = pg_conn.cursor()

In [6]:
table_create_sql = '''
CREATE TABLE IF NOT EXISTS Articles (
    id INT PRIMARY KEY,
    title TEXT,
    url TEXT,
    domain TEXT,
    type TEXT,
    cleaned_content TEXT,
    scraped_at TIMESTAMP,
    inserted_at TIMESTAMP,
    updated_at TIMESTAMP,
    meta_description TEXT,
    summary TEXT
);

CREATE TABLE IF NOT EXISTS Authors (
    author_name TEXT,
    article_id INT,
    PRIMARY KEY (author_name, article_id),
    FOREIGN KEY (article_id) REFERENCES Articles(id)
);

CREATE TABLE IF NOT EXISTS Keywords (
    keyword_name TEXT,
    article_id INT,
    PRIMARY KEY (keyword_name, article_id),
    FOREIGN KEY (article_id) REFERENCES Articles(id)
);

CREATE TABLE IF NOT EXISTS Meta_Keywords (
    meta_keyword_name TEXT,
    article_id INT,
    PRIMARY KEY (meta_keyword_name, article_id),
    FOREIGN KEY (article_id) REFERENCES Articles(id)
);

CREATE TABLE IF NOT EXISTS Tags (
    tag_name TEXT,
    article_id INT,
    PRIMARY KEY (tag_name, article_id),
    FOREIGN KEY (article_id) REFERENCES Articles(id)
);
'''

cur.execute(table_create_sql)

In [7]:
trigger_create_sql = '''
CREATE OR REPLACE FUNCTION pass() RETURNS trigger AS $$
    BEGIN
        RETURN NULL;
    END;
$$ LANGUAGE plpgsql ;

CREATE TRIGGER InsertedNotNull
    BEFORE INSERT ON Authors
    FOR EACH ROW
    WHEN ( NEW.author_name IS NULL )
    EXECUTE PROCEDURE pass();

CREATE TRIGGER InsertedNotNull
    BEFORE INSERT ON Keywords
    FOR EACH ROW
    WHEN ( NEW.keyword_name IS NULL )
    EXECUTE PROCEDURE pass();

CREATE TRIGGER InsertedNotNull
    BEFORE INSERT ON Meta_Keywords
    FOR EACH ROW
    WHEN ( NEW.meta_keyword_name IS NULL )
    EXECUTE PROCEDURE pass();

CREATE TRIGGER InsertedNotNull
    BEFORE INSERT ON Tags
    FOR EACH ROW
    WHEN ( NEW.tag_name IS NULL )
    EXECUTE PROCEDURE pass();
'''

cur.execute(trigger_create_sql)

In [8]:
postgres_data_path = "/postgres_data/"

database_articles.to_csv(postgres_data_path + 'database_articles.csv', index=False)
database_authors.to_csv(postgres_data_path + 'database_authors.csv', index=False)
database_keywords.to_csv(postgres_data_path + 'database_keywords.csv', index=False)
database_meta_keywords.to_csv(postgres_data_path + 'database_meta_keywords.csv', index=False)
database_tags.to_csv(postgres_data_path + 'database_tags.csv', index=False)

In [9]:
copy_sql = """
COPY Articles(id, domain, type, url, cleaned_content, scraped_at, inserted_at, updated_at, title, meta_description, summary)
FROM '""" + postgres_data_path + """database_articles.csv'
DELIMITER ','
CSV HEADER;

COPY Authors(article_id, author_name) FROM '""" + postgres_data_path + """database_authors.csv'
DELIMITER ','
CSV HEADER ;

COPY Keywords(article_id, keyword_name) FROM '""" + postgres_data_path + """database_keywords.csv'
DELIMITER ','
CSV HEADER ;

COPY Meta_Keywords(article_id, meta_keyword_name) FROM '""" + postgres_data_path + """database_meta_keywords.csv'
DELIMITER ','
CSV HEADER ;

COPY Tags(article_id, tag_name) FROM '""" + postgres_data_path + """database_tags.csv'
DELIMITER ','
CSV HEADER ;
"""

cur.execute(copy_sql)

In [10]:
pg_conn.commit()

# Task 1
The database schemas are designed with the objective of reducing redundancy and accomidating future extensions of the database. In particular, we note that the articles of FakeNewsCorpus dataset in some cases have several authors. If authors were to part of a articles schema, there would be significant redundency in the database, considering that tuples would exist with identicial information for all but the author attribute. Therefore, a seperate schema for authors have been created, additionaly making all attributes atomic data types, which helps avoid repeating groups and consequently, in some scenarios, inefficient queries.
Similar arguments holds for keywords, meta keywords and tags for articles in the dataset, hence, the choice of seperating them into their own database schemas consiting of atomic attributes only. In turn, this aspect of database normilization allows the database to be extended upon without neccesarily changing the exisiting structure.

One downside of our choice of realations is that we in some cases will have several tuples cointaing information about the same entity. Using the Authors relation as an example, each author will have a tuple for each article they have authored. If no author names are the same, and names are always the same, this is not a problem. However, if this is not the case, it could cause several problems with maintaining information about authors.

The first simple problem, is that of update anomalies. If an author changes name, we must make sure to change his name in all tuples containing his name, so it is clear these articles still have the same author.

A more complicated issue is the problem of entity resolution. Using this design it can be hard to tell which articles actually have the same author. Some authors could have the same name, and some names could be spelled diferently in diferent articles, even though the author was the same.

One way to combat this issue would be to split the relation into two relations. One relation containing author names, and some form of unique id, and another containing tuples of author ids and article ids showing the authors of each article. However, such a database would require more data, computation and space to maintain, and we therefore choose to go with the simpler design.

In [11]:
query = '''
SELECT count(*) FROM articles;
'''

print(psql.read_sql(query, pg_conn))

    count
0  100000


# Task 2

We created the following relations for our database:

* Articles
* Authors
* Keywords
* Meta_Keywords
* Tags

The Articles relation contains the main information about the articles, and has the attributes "id", "domain", "type", "url", "content", "scraped_at", "inserted_at", "updated_at", "title", "meta_description", "summary".
The single primary key for this relation is the "id", which therefore functionally determines all other attributes.

The Authors relation contains tuples of authors and articles wrtitten by these authors, referenced by the "id" attribute of the articles. The relation therefore has the attributes "article_id" and "author_name", the combination of which form the primary key. There are therefore no nontrivial functoinal dependencies.

The Keywords relation contains the attributes "article_id" and "keyword_name". As with the Authors relation, both attributes together form the primary key for the relation, and there are therefore no nontrivial functional dependencies.

The Meta_Keywords relation has similar structure, and contains the attributes "article_id" and "meta_keyword_name". Since both attributes form the primary key, there are no nontrivial functional dependencies.

Finally, the Tags relation also has a similar structure, with the attributes and also the primary key being "article_id" and "tag_name", resulting in no nontrivial functional dependencies.

All relations therefore respect the BCNF condiction and are in BCNF form.

Looking at the data, it may seem like domains uniquly determine the type of article, which would violate the BCNF condition. We, however, would argue that this is not neccesarily the case for all data that could be inserted in the database, as we believe there could be sites producing both real and fake news. We have therefore choosen to stick with this database design, even though it causes some redundancy with the given data.

# Task 3

1. List the domains of news articles of reliable type and scraped at or after January 15, 2018. NOTE: Do not include duplicate domains in your answer. [Languages: relational algebra and SQL]

**Relational Algebra:**

$$ \pi_{domain}\left(\sigma_{type ~=~ 'reliable' ~\land~ scraped\_at ~\geq~ '2018-01-15 00:00:00.0'}(R_{articles})\right) $$

**SQL:
**
```sql
SELECT DISTINCT domain FROM articles
WHERE type = 'reliable' AND scraped_at >= '2018-01-15 00:00:00.0';
```

**Observation(s):**
Executing the query on our database yields the following domains:

`christianpost.com, consortiumnews.com`

This shows that only two news sources have produced articles of *reliable* type at or before January 15, 2018. Considering that there are a total of 246 different domains in our dataset, it is a low proportion of news sources, at least in that time frame, producing articles that are *reliable*.

In [12]:
query = '''
SELECT DISTINCT domain FROM articles
WHERE type = 'reliable' AND scraped_at >= '2018-01-15 00:00:00.0';
'''

print(psql.read_sql(query, pg_conn))

               domain
0   christianpost.com
1  consortiumnews.com


2. List the name(s) of the most prolific author(s) of news articles of fake type. An author is among the most prolific if it has authored as many or more fake news articles as any other author in the dataset. [Languages: extended relational algebra and SQL]

**Relational Algebra:**

$$ P := \gamma_{author\_name, COUNT(id)}(R_{articles} \bowtie R_{authors}) $$
$$ Q := \pi_{MAX(count)}(P) $$
$$ \pi_{author\_name}\left(\sigma_{count ~\in~ Q}(P)\right) $$

**SQL:**

```sql
WITH P as (
    SELECT a.author_name, count(*) count FROM Articles
    JOIN authors a on articles.id = a.article_id
    WHERE type = 'fake'
    GROUP BY a.author_name
)

SELECT author_name, count FROM P
WHERE count IN (
    SELECT max(count) from P
);
```

Observation(s):
Executing the query on our database tells us that the author John Rolls has created the most fake news articles. In fact the database contains a total of 876 fake news articles from him.

In [13]:
query = '''
WITH P as (
    SELECT a.author_name, count(*) count FROM Articles
    JOIN authors a on articles.id = a.article_id
    WHERE type = 'fake'
    GROUP BY a.author_name
)

SELECT author_name, count FROM P
WHERE count IN (
    SELECT max(count) from P
);
'''

print(psql.read_sql(query, pg_conn))

  author_name  count
0  john rolls    876


3. Count the pairs of article IDs that exhibit the exact same set of meta-keywords, but only return the pairs where the set of meta-keywords is not empty. [Language: SQL]

SQL:

```sql
SELECT count(*) FROM articles a1
CROSS JOIN articles a2
WHERE a1.id < a2.id
AND EXISTS(
    SELECT mk1.meta_keyword_name FROM meta_keywords mk1
    WHERE mk1.article_id = a1.id
)
AND NOT EXISTS(
    SELECT mk1.meta_keyword_name FROM meta_keywords mk1
    WHERE mk1.article_id = a1.id
    EXCEPT (
        SELECT mk2.meta_keyword_name FROM meta_keywords mk2
        WHERE mk2.article_id = a2.id
    )
);
```

Observation(s):
We were not able to finish this query, likely due to the astronomical number of pairs that resulted from the join ($\binom{100000}{2} \approx 5 * 10^{11}$). When running on a smaller subset of data (first 1000 rows), there were 28 pairs of articles that matched the criteria.

In [14]:
query = '''
SELECT count(*) FROM articles a1
CROSS JOIN articles a2
WHERE a1.id < a2.id
AND EXISTS(
    SELECT mk1.meta_keyword_name FROM meta_keywords mk1
    WHERE mk1.article_id = a1.id
)
AND NOT EXISTS(
    SELECT mk1.meta_keyword_name FROM meta_keywords mk1
    WHERE mk1.article_id = a1.id
    EXCEPT (
        SELECT mk2.meta_keyword_name FROM meta_keywords mk2
        WHERE mk2.article_id = a2.id
    )
);
'''

#print(psql.read_sql(query1, pg_conn))

# Task 4

An interesting, almost neccesary query, is that of the proportion of articles of the various types, which the following queuery can answer.

From this queuery we see that the dataset contains a disproportionate amount of articles of *fake* type - approximately $46\%$.

In [15]:
query = '''
SELECT type, count(*) FROM articles
GROUP BY type;
'''

print(psql.read_sql(query, pg_conn))

          type  count
0         bias   9123
1    clickbait   1979
2   conspiracy   6877
3         fake  45768
4         hate    298
5      junksci   2204
6    political  27371
7     reliable    289
8        rumor    124
9       satire    344
10     unknown   1067
11  unreliable   1045
12        None   3511


For further analysis, in respect to fake news detection using these exact articles, it would important to know the volume of data used to learn characteristics of the data, and,perhaps, also the number of unique keywords, meta keywords and tags describing different article types. This is explored executing the following queries.

Firstly, we note that the database does not contain any keywords and that not all articles type has meta keywords describing them (e.g. *hate*). The queries with respect to meta keywords tells us that articles of type *reliable* and *fake* has significantly less distinct meta keywords describing them, than for instances, articles of type *conspiracy* and *political*. Out of a total of $29418$ distinct meta keywords, 16542 meta keywords, corresponding to roughly $56\%$, are used to represent articles of political type and only $\approx .04$ and $\approx 0.7\%$ for articles of type *reliable* and *fake*, respectively.

In regards to the set of tags describing different article types, it is seen that article types *rumor*, *hate* and *reliable* has $0.11\%$, $0.18\%$ and $0.32\%$ of the total $36876$ distinct tags, respectively. Analogus to meta keywords, articles of *political* type has highest proportion of tags describing it, that is here roughly $33\%$.

Further interesing and supporting observations could be that of the average amount of meta keywords and tags describing the different article types.

In [16]:
def get_counts(name, relation, title):
    query1 = '''
    SELECT count(distinct ''' + name + ''') FROM ''' + relation + ''';
    '''
    
    query2 = '''
    SELECT type, count(distinct ''' + name + ''') count FROM ''' + relation + '''
    JOIN articles a ON a.id = ''' + relation + '''.article_id
    GROUP BY type
    ORDER BY count;
    '''
    
    print(title + ':')
    print(psql.read_sql(query1, pg_conn))
    print(psql.read_sql(query2, pg_conn))
    print()

get_counts('keyword_name', 'keywords', "KEYWORDS")
get_counts('meta_keyword_name', 'meta_keywords', "META_KEYWORDS")
get_counts('tag_name', 'tags', "TAGS")

KEYWORDS:
   count
0      0
Empty DataFrame
Columns: [type, count]
Index: []

META_KEYWORDS:
   count
0  29418
          type  count
0     reliable     12
1         fake     22
2    clickbait    127
3       satire    262
4   unreliable    469
5      unknown    662
6      junksci   3235
7         None   4199
8         bias   4681
9   conspiracy   5876
10   political  16542

TAGS:
   count
0  36876
          type  count
0        rumor     41
1         hate     65
2     reliable    119
3   unreliable    715
4         None    761
5      junksci    824
6       satire    911
7      unknown   1078
8         fake   3218
9         bias   6598
10   clickbait   8452
11  conspiracy  12355
12   political  12607



A final observation one can make using SQL-queries is the average amount of authors of articles of different types, as answered by the below query.

It can be seen that fake articles have the lowest amount of authors, which was also our hypothesis.

Further, we can see that reliable articles have a large amount of authors on average, only beaten by hate-articles.

The fact that hate-articles have the largest amount of authors is a bit surprising, and would warrant further investigation into whether this is due to artifacts or is actually consistent with reality.

In [17]:
query = '''
SELECT type, avg(count) avg FROM articles
JOIN (
    SELECT article_id, count(author_name) count FROM authors
    GROUP BY article_id
) author_counts
ON article_id = id
GROUP BY type
ORDER BY avg;
'''

print(psql.read_sql(query, pg_conn))

          type       avg
0         fake  1.030210
1         None  1.190649
2        rumor  1.435897
3    clickbait  2.124850
4   conspiracy  2.433333
5       satire  2.487500
6   unreliable  3.270199
7      junksci  3.476415
8         bias  4.162914
9      unknown  4.223118
10   political  4.651550
11    reliable  5.746835
12        hate  7.580769


In [18]:
cur.close()